# Code taken from

https://www.youtube.com/watch?v=kCc8FmEb1nY

https://github.com/karpathy/ng-video-lecture

# Import Torch and define hyperparameters

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 8 # how many independent sequences will we process in parallel?
block_size = 64 # what is the maximum context length for predictions?
max_iters = 500
eval_interval = 100
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 25
n_embd = 64
n_head = 6
n_layer = 2
dropout = 0.05
# ------------

torch.manual_seed(1337)

# Read the input text

In [3]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# Create a naive tokenizer

In [4]:
# Get all the unique characters from the text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# Create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train-test-split data

In [5]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# Define the GPT model step by step

## First, define some helpers

In [6]:
# A simple data loading function
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

# Function to calculate the loss
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


# A single feed-forward neural network
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

## Second, define a single attention head

## Assignment: fill in the blanks below

In [7]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        q = self.query(x)
        k = self.key(x)
        kT = k.transpose(-2,-1)
        dk = k.shape[-1]
        v = self.value(x)
        # Assignment: fill in the blanks to compute attention scores
        # Remember the lecture, there are two steps missing below
        attention_scores = q @ kT
        attention_scores = attention_scores * dk ** (-0.5)
        # create a mask for the decoder
        B,T,C = x.shape
        attention_scores = attention_scores.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        # Assignment: apply the softmax function
        attention_scores = F.softmax(attention_scores)
        attention_scores = self.dropout(attention_scores) # An optional Dropout layer
        # perform the weighted aggregation of the values
        # Assignment: apply the final step in the attention mechanism
        attention_scores = attention_scores @ v
        return attention_scores

## Third, combine several attention heads into multi-head attention

In [8]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

## Create a single Transformer block

In [9]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.self_attention = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x_normed1 = self.ln1(x)
        attention_out = self.self_attention(x_normed1)
        x = x + x_normed1
        x_normed2 = self.ln2(x)
        feed_forward_out = self.ffwd(x_normed2)
        x = x + feed_forward_out
        return x

## Fifth, combine everything into the GPT model

## Assignment: fill in the blanks below

In [12]:
class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # Define embedding tables for the tokens and position embeddigns
        # each token directly reads off the values for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        # Define a sequence of several self_attention blocks
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        # Define the final normalization layer and one single neural network layer (called Linear because it's just a linear neural-network model with a single mapping from an input layer to the output layer, no hidden layers)
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        # Assignment: complete the transformer steps below.
        x = tok_emb + pos_emb # 1. Add the input embeddings
        x = self.blocks(x) # 2. Run the attention mechanism. Hint: Use one of the objects defined in _init_()
        x = self.ln_f(x) # 3. Run the final normalization. Hint: Use one of the objects defined in _init_()
        logits = self.lm_head(x) # 4. Run the output through a final linear neural network model.  Hint: Use one of the objects defined in _init_() ... # 4. Run the output through a final linear neural network model.  Hint: Use one of the objects defined in __init__()

        # Compute the loss
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, _ = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [11]:
model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))
#open('more.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))


0.110145 M parameters


/var/folders/wj/tvdm7qp11rlf5k20r5kb0qg80000gq/T/ipykernel_33740/2000775134.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  attention_scores = F.softmax(attention_scores)


step 0: train loss 4.1740, val loss 4.1736
step 100: train loss 3.0505, val loss 3.1091
step 200: train loss 2.7555, val loss 2.7605
step 300: train loss 2.6330, val loss 2.6361
step 400: train loss 2.5827, val loss 2.5997
step 499: train loss 2.5683, val loss 2.5614

Iurinemey on irumee,,:
O:
Vir oroder tooul fafo iwyo$ounimerid? y t yoveog
T. :
xs tthow whaweryofrs! chte, hil a baicranNLORAsathiy t lle Hensx, wr, tre s wo aprest'?

'd we sd atyo Wh d.
T: iamyonde oundotol
AFs mupl,
TI d thaf test he oriKFis ithed ea figaizy pd f Wh pot mmucede se d
IRD a h, nd s
LIBWe be fra s;
IArthae meaf ly avuathat atre! ttlllis gt.

An eBuf asterzyyaY:
D faldtheroouGL:
fou ofandsv,
Yanjin theatedld:
ANe wnthe y f bum sous thers IK;

LCeveldo vishlitent seysuQRT:
fle
Pi
